In [44]:
import pandas as pd
import fastparquet


In [45]:
df = pd.read_parquet('tiling_results.parquet')
df.shape

(10682, 16)

In [46]:
len(df[df.tiling_compatible == True]) 

524

In [47]:
counts = df['incompatible_reason'].value_counts(dropna=False).reset_index()
counts.columns = ['Value', 'Count']
print(counts)

                    Value  Count
0      unsupported_format   5009
1         group_structure   1212
2  tile_generation_failed   1107
3   failed_to_extract_url    962
4        no_xy_dimensions    886
5          cant_open_file    866
6                    None    546
7  cant_extract_variables     94


In [52]:
df[df.incompatible_reason == 'tile_generation_failed'].error_message.unique()


array(['list index out of range',
       'Expecting value: line 1 column 1 (char 0)',
       "b'CDF\\x01\\x00\\x00\\x00\\x00' is not the signature of a valid netCDF4 file",
       'titiler.xarray can only work with 2D or 3D dataset',
       'Unable to allocate 15.6 GiB for an array with shape (12, 13165, 26477) and data type float32',
       'No assets found for tile 0-0-0',
       "b'CDF\\x01\\x00\\x00\\x01m' is not the signature of a valid netCDF4 file",
       "b'CDF\\x01\\x00\\x00\\x00\\x0c' is not the signature of a valid netCDF4 file",
       "b'CDF\\x01\\x00\\x00\\x00\\xf0' is not the signature of a valid netCDF4 file",
       "No group found in store <fsspec.mapping.FSMap object at 0x7f554a6ee5d0> at path ''",
       "b'CDF\\x01\\x00\\x00M\\x19' is not the signature of a valid netCDF4 file",
       'Assets HAVE TO have the same number of bands',
       "b'CDF\\x01\\x00\\x00\\x13\\x08' is not the signature of a valid netCDF4 file",
       '"No variable named \'altitude\'. Variab

In [56]:
list(df[df.error_message == "No group found in store <fsspec.mapping.FSMap object at 0x7f51a953e270> at path ''"].collection_concept_id)

['C1979862427-GHRC_DAAC']

In [33]:
# Filter out None/NaN and empty arrays
subset_df = df[df['data_variables'].apply(
    lambda x: x is not None and (isinstance(x, np.ndarray) and (x.size > 0 and x[0] != None))
)]
subset_df.data_variables

27                                               [SWIR1]
29                                               [TIRS2]
34     [latitude, longitude, scan_datetime, year, mon...
37     [aerosol_optical_thickness_550_land, aerosol_o...
43     [Aerosol_Optical_Thickness_550_Expected_Uncert...
                             ...                        
623    [Delta_Time, SZA, VZA, SAz, VAz, Longitude, La...
625                                             [weight]
640    [obs_id, fov_obs_id, obs_time_tai93, obs_time_...
643    [EFLUXICE, EFLUXWTR, FRSEAICE, HFLUXICE, HFLUX...
646                    [OriginalInputFiles, XmlMetadata]
Name: data_variables, Length: 160, dtype: object

In [47]:
all_unique_vars = set().union(*subset_df.data_variables)

keywords = ['time', 'latitude', 'longitude', 'num_obs', 'foo', 'crs']
filtered_list = [item for item in all_unique_vars if not any(keyword in item.lower() for keyword in keywords)]
print(len(filtered_list))

509


In [36]:
with open('titiler_cmr_compatibility/known_vars.txt', 'w') as f:
    for item in filtered_list:
        f.write(f"{item}\n")

In [11]:
# !aws s3 cp s3://nasa-veda-scratch/aimee/tiling_results.parquet \
#     s3://nasa-veda-scratch/aimee/tiling_results_11-08-2025.parquet

copy: s3://nasa-veda-scratch/aimee/tiling_results.parquet to s3://nasa-veda-scratch/aimee/tiling_results_11-08-2025.parquet
